In [170]:
import numpy as np
import pandas as pd
import graphviz
import dowhy
from dowhy import CausalModel
from sklearn.preprocessing import LabelEncoder

In [171]:
path_to_file= "C:/Users/thack/OneDrive/Desktop/BPIC-2015/BPIC15_1_sorted.csv"
df=pd.read_csv(path_to_file,dtype={'dueDate':str,'dateStop':str,'question':str})

In [172]:
#all closed events
closed=['G']
df=df[df['(case) caseStatus'].isin(closed)]

#only permit denied(geweugerd) or permit issued(verleend)
result=['Vergunning verleend']
df=df[df['(case) last_phase'].isin(result)]

#only Building permits
typ=['Bouw']
df=df[df['(case) parts'].isin(typ)]
df

,Case ID,Activity,Resource,Complete Timestamp,(case) IDofConceptCase,(case) Includes_subCases,(case) Responsible_actor,(case) SUMleges,(case) caseProcedure,(case) caseStatus,...,activityNameEN,activityNameNL,concept:name,dateFinished,dateStop,dueDate,lifecycle:transition,monitoringResource,planned,question
36,3399416,01_BB_540\\complete,2670601,7/8/11 12:31:40,3400078.0,N,4901428.0,866.73890,NaN,G,...,objection lodged against decision,bezwaar tegen beschikking ingediend,01_BB_540,07/08/11 12:31,NaN,NaN,complete,560464,2011-05-24T09:59:14+02:00,FALSE
50,3833827,01_BB_540\\complete,2670601,11/7/11 15:20:33,NaN,J,4901428.0,399.58200,NaN,G,...,objection lodged against decision,bezwaar tegen beschikking ingediend,01_BB_540,11/07/11 15:20,NaN,NaN,complete,2670601,2011-11-08T15:20:30+01:00,FALSE
54,4119215,01_BB_540\\complete,560464,4/10/12 10:36:18,4120038.0,J,4901428.0,1444.85140,NaN,G,...,objection lodged against decision,bezwaar tegen beschikking ingediend,01_BB_540,04/10/12 10:36,NaN,NaN,complete,560890,2012-02-15T08:57:10+01:00,FALSE
284,3399416,01_BB_770\\complete,2670601,7/8/11 12:31:40,3400078.0,N,4901428.0,866.73890,NaN,G,...,set phase: phase permitting irrevocable,instellen besluitfase: oorspronkelijk besluit,01_BB_770,07/08/11 12:31,NaN,NaN,complete,560464,2011-07-09T12:31:41+02:00,EMPTY
299,3833827,01_BB_770\\complete,2670601,11/7/11 15:20:33,NaN,J,4901428.0,399.58200,NaN,G,...,set phase: phase permitting irrevocable,instellen besluitfase: oorspronkelijk besluit,01_BB_770,11/07/11 15:20,NaN,NaN,complete,2670601,2011-11-08T15:20:33+01:00,EMPTY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52178,3645961,15_NGV_010\\complete,3273854,9/5/11 9:50:07,3646104.0,J,4901428.0,1309.59200,NaN,G,...,request further information,nadere gegevens vragen,15_NGV_010,09/05/11 10:36,NaN,NaN,complete,3273854,NaN,FALSE
52185,3704488,15_NGV_010\\complete,2670601,8/30/11 14:38:53,3704631.0,J,4901428.0,4927.01460,NaN,G,...,request further information,nadere gegevens vragen,15_NGV_010,08/30/11 14:40,NaN,NaN,complete,2670601,NaN,FALSE
52188,3720299,15_NGV_010\\complete,2670601,9/15/11 12:00:23,NaN,J,4901428.0,3367.92000,NaN,G,...,request further information,nadere gegevens vragen,15_NGV_010,09/15/11 12:07,NaN,NaN,complete,2670601,NaN,FALSE
52190,3728500,15_NGV_010\\complete,2670601,10/19/11 15:49:40,3729523.0,J,4901428.0,1436.10110,NaN,G,...,request further information,nadere gegevens vragen,15_NGV_010,10/19/11 15:49,NaN,NaN,complete,2670601,NaN,FALSE


In [173]:
#df2=Case-Attribut-Table
df2=df.copy()
df2=df.drop(['Activity','(case) parts','(case) last_phase','Resource','Complete Timestamp','(case) case_type','(case) termName','(case) IDofConceptCase','(case) caseStatus','activityNameNL','concept:name','dateFinished','dateStop','dueDate','lifecycle:transition','planned','(case) caseProcedure','activityNameEN','action_code','monitoringResource','question','(case) landRegisterID','(case) Responsible_actor'],axis=1)
df2=df2.drop_duplicates()
df2

,Case ID,(case) Includes_subCases,(case) SUMleges,(case) requestComplete
36,3399416,N,866.73890,True
50,3833827,J,399.58200,False
54,4119215,J,1444.85140,True
699,2771451,J,1354.25420,True
701,2783345,J,2302.04760,True
...,...,...,...,...
1208,4775758,J,339.56040,True
1210,4784091,J,1153.61180,True
1211,4785864,J,379.22354,True
1216,4811353,J,-195.19666,True


In [174]:
#convert binary columns to int
lE = LabelEncoder()
l1 = lE.fit_transform(df2['(case) Includes_subCases'])
l1
df2["(case) Includes_subCases"] = l1

lE = LabelEncoder()
l1 = lE.fit_transform(df2['(case) requestComplete'])
l1
df2["(case) requestComplete"] = l1

In [175]:
#df3= Event-Attribut-Table
df3=df.drop(['Activity','(case) Includes_subCases','monitoringResource','(case) case_type','(case) IDofConceptCase','(case) termName','(case) caseStatus','activityNameNL','concept:name','dateFinished','dateStop','dueDate','lifecycle:transition','planned','(case) termName','(case) caseProcedure','activityNameEN','(case) Responsible_actor','(case) requestComplete','(case) SUMleges','(case) landRegisterID','(case) last_phase','(case) parts','(case) case_type','question'],axis=1)

df3['Complete Timestamp'] = df3['Complete Timestamp'].astype('datetime64')
df3['action_code'] = df3['action_code'].astype('string')

events=['01_HOOFD_010','01_HOOFD_015','01_HOOFD_020','01_HOOFD_065_1','01_HOOFD_065_2','01_HOOFD_110','01_HOOFD_120','01_HOOFD_180','01_HOOFD_200','01_HOOFD_250','01_HOOFD_260','09_AH_I_010','01_HOOFD_370','01_HOOFD_375','01_HOOFD_380','01_HOOFD_430','01_HOOFD_480','01_HOOFD_490','01_HOOFD_490_2']
df3=df3[df3['action_code'].isin(events)]
df3=df3.sort_values(by=['Case ID','Complete Timestamp'])
df3

,Case ID,Resource,Complete Timestamp,action_code
699,2771451,6,2010-10-06 00:00:00,01_HOOFD_010
10215,2771451,6,2010-11-24 00:00:00,01_HOOFD_065_2
2692,2771451,6,2010-11-30 16:21:56,01_HOOFD_015
3889,2771451,6,2010-11-30 16:22:02,01_HOOFD_020
9564,2771451,6,2010-11-30 16:50:30,01_HOOFD_065_1
...,...,...,...,...
23385,4824215,3273854,2012-04-05 15:56:52,01_HOOFD_375
24315,4824215,3273854,2012-04-05 15:56:52,01_HOOFD_380
25225,4824215,3273854,2012-04-05 15:56:53,01_HOOFD_430
27072,4824215,3273854,2012-04-05 15:56:55,01_HOOFD_480


In [176]:
#StartTime per case table
event=['01_HOOFD_010']

df4=df3[df3['action_code'].isin(event)]
df4.rename(columns={'Complete Timestamp':'StartTime'}, inplace=True)

df4=df4.drop(['Resource','action_code'],axis=1)
df4

C:\Users\thack\AppData\Local\Temp\ipykernel_13844\2057451376.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4.rename(columns={'Complete Timestamp':'StartTime'}, inplace=True)


,Case ID,StartTime
699,2771451,2010-10-06
701,2783345,2010-10-07
725,2857143,2010-11-09
728,2868714,2010-11-15
733,2875758,2010-11-20
...,...,...
1208,4775758,2012-03-19
1210,4784091,2012-03-20
1211,4785864,2012-03-23
1216,4811353,2012-03-27


In [177]:
#Calculate dayspast start to get a int time variable per event
df3=df3.merge(df4,how='left', left_on='Case ID', right_on='Case ID')
for index, row in df3.iterrows():
 df3.at[index,'Complete Timestamp']=(df3.at[(index),'Complete Timestamp']-df3.at[index,'StartTime']).days
df3=df3.drop(['StartTime'],axis=1)
df3.rename(columns={'Complete Timestamp':'dayspast'}, inplace=True)
df3['dayspast']=df3['dayspast'].astype('int')
df3

,Case ID,Resource,dayspast,action_code
0,2771451,6,0,01_HOOFD_010
1,2771451,6,49,01_HOOFD_065_2
2,2771451,6,55,01_HOOFD_015
3,2771451,6,55,01_HOOFD_020
4,2771451,6,55,01_HOOFD_065_1
...,...,...,...,...
2428,4824215,3273854,6,01_HOOFD_375
2429,4824215,3273854,6,01_HOOFD_380
2430,4824215,3273854,6,01_HOOFD_430
2431,4824215,3273854,6,01_HOOFD_480


In [178]:
#remove event duplicates in case so we can flatten the table without getting column with many NaN values
df3=df3.sort_values(by=['Case ID','action_code','dayspast'])
df3=df3.drop_duplicates(subset=['Case ID','action_code'], keep='last')
df3
    

,Case ID,Resource,dayspast,action_code
0,2771451,6,0,01_HOOFD_010
2,2771451,6,55,01_HOOFD_015
3,2771451,6,55,01_HOOFD_020
4,2771451,6,55,01_HOOFD_065_1
1,2771451,6,49,01_HOOFD_065_2
...,...,...,...,...
2429,4824215,3273854,6,01_HOOFD_380
2430,4824215,3273854,6,01_HOOFD_430
2431,4824215,3273854,6,01_HOOFD_480
2432,4824215,560890,11,01_HOOFD_490_2


In [179]:
#transform action_code to int
lE = LabelEncoder()
l1 = lE.fit_transform(df3['action_code'])
l1
df3["action_code"] = l1
df3

,Case ID,Resource,dayspast,action_code
0,2771451,6,0,0
2,2771451,6,55,1
3,2771451,6,55,2
4,2771451,6,55,3
1,2771451,6,49,4
...,...,...,...,...
2429,4824215,3273854,6,13
2430,4824215,3273854,6,14
2431,4824215,3273854,6,15
2432,4824215,560890,11,16


In [180]:
#flatten table
df3=df3.sort_values(by=['Case ID','dayspast'])
cc = df3.groupby(['Case ID']).cumcount() + 1
df3 = df3.set_index(['Case ID', cc]).unstack().sort_index(1, level=1)
df3.columns = ['_'.join(map(str,i)) for i in df3.columns]
df3.reset_index()
df3

C:\Users\thack\AppData\Local\Temp\ipykernel_13844\35637081.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_index will be keyword-only.
  df3 = df3.set_index(['Case ID', cc]).unstack().sort_index(1, level=1)


,Resource_1,action_code_1,dayspast_1,Resource_2,action_code_2,dayspast_2,Resource_3,action_code_3,dayspast_3,Resource_4,...,dayspast_15,Resource_16,action_code_16,dayspast_16,Resource_17,action_code_17,dayspast_17,Resource_18,action_code_18,dayspast_18
Case ID,,,,,,,,,,,,,,,,,,,,,
2771451,6,0,0,6,4,49,6,1,55,6,...,56,2670601,15,56,2670601,17,56,560890,16,61
2783345,2670601,0,0,2670601,4,4,2670601,1,55,2670601,...,55,2670601,15,55,2670601,17,55,560890,16,60
2857143,560925,0,0,560925,1,2,560925,2,2,560925,...,35,560890,15,35,560890,16,35,560890,17,35
2868714,560872,0,0,560872,1,3,560872,2,3,560872,...,64,560890,15,64,560890,16,64,560890,17,64
2875758,560925,0,0,560925,1,3,560925,2,3,560925,...,52,560894,15,52,560894,16,52,560894,17,52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4775758,560872,0,0,560872,1,3,560872,2,3,560872,...,29,2670601,15,29,2670601,16,29,2670601,17,29
4784091,560872,0,0,560872,1,6,560872,2,6,560872,...,44,2670601,15,44,2670601,17,44,2670601,16,48
4785864,560872,0,0,560872,1,3,560872,2,3,560872,...,25,3273854,15,25,3273854,17,25,560890,16,31


In [181]:
#merge case table with flattened event table
df3=df3.merge(df2,how='left', left_on='Case ID', right_on='Case ID')
df3=df3.drop(['Case ID'],axis=1)

In [182]:
df3.rename(columns={'(case) Includes_subCases':'Includes_subCases'}, inplace=True)
df3.rename(columns={'(case) SUMleges':'SUMleges'}, inplace=True)
df3.rename(columns={'(case) requestComplete':'requestComplete'}, inplace=True)

In [183]:
# We drop AC: 3 R: 10, 13, 14,16 D: 13 14 due to Singular matrix error
df4=df3.drop(['action_code_3','Resource_10','Resource_13','dayspast_13','Resource_14','dayspast_14','Resource_16'],axis=1)

df4

,Resource_1,action_code_1,dayspast_1,Resource_2,action_code_2,dayspast_2,Resource_3,dayspast_3,Resource_4,action_code_4,...,dayspast_16,Resource_17,action_code_17,dayspast_17,Resource_18,action_code_18,dayspast_18,Includes_subCases,SUMleges,requestComplete
0,6,0,0,6,4,49,6,55,6,2,...,56,2670601,17,56,560890,16,61,0,1354.25420,1
1,2670601,0,0,2670601,4,4,2670601,55,2670601,2,...,55,2670601,17,55,560890,16,60,0,2302.04760,1
2,560925,0,0,560925,1,2,560925,2,560925,3,...,35,560890,16,35,560890,17,35,0,4550.37900,1
3,560872,0,0,560872,1,3,560872,3,560872,3,...,64,560890,16,64,560890,17,64,0,241.32560,1
4,560925,0,0,560925,1,3,560925,3,560925,3,...,52,560894,16,52,560894,17,52,0,148.36800,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,560872,0,0,560872,1,3,560872,3,560872,3,...,29,2670601,16,29,2670601,17,29,0,339.56040,1
126,560872,0,0,560872,1,6,560872,6,560872,3,...,44,2670601,17,44,2670601,16,48,0,1153.61180,1
127,560872,0,0,560872,1,3,560872,3,560872,3,...,25,3273854,17,25,560890,16,31,0,379.22354,1
128,560925,0,0,560925,1,2,560925,2,560925,3,...,23,3273854,17,23,560890,16,24,0,-195.19666,1


In [184]:
from causallearn.search.ConstraintBased.PC import (get_adjacancy_matrix,mvpc_alg, pc, pc_alg)
from causallearn.utils.cit import chisq, fisherz, gsq, kci, mv_fisherz
from causallearn.utils.GraphUtils import GraphUtils
from causallearn.search.ConstraintBased.PC import pc
from IPython.display import Image, display
def view_pydot(pdot):
    plt = Image(pdot.create_png())
    display(plt)

In [185]:
cg = pc(df4.to_numpy(),0.05, mv_fisherz, True, 0,0)

C:\Users\thack\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:2829: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\thack\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:2830: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


  0%|          | 0/50 [00:00<?, ?it/s]

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\thack\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\thack\AppData\Local\Temp\ipykernel_13844\1693584986.py", line 1, in <cell line: 1>
    cg = pc(df4.to_numpy(),0.05, mv_fisherz, True, 0,0)
  File "C:\Users\thack\AppData\Local\Programs\Python\Python310\lib\site-packages\causallearn\search\ConstraintBased\PC.py", line 45, in pc
    return pc_alg(data=data, node_names=node_names, alpha=alpha, indep_test=indep_test, stable=stable, uc_rule=uc_rule,
  File "C:\Users\thack\AppData\Local\Programs\Python\Python310\lib\site-packages\causallearn\search\ConstraintBased\PC.py", line 102, in pc_alg
    cg_1 = SkeletonDiscovery.skeleton_discovery(data, alpha, indep_test, stable,
  File "C:\Users\thack\AppData\Local\Programs\Python\Python310\lib\site-packages\causallearn\utils\PCUtils\SkeletonDiscovery.py", li

In [186]:
dot = GraphUtils.to_pydot(cg.G, labels=df4.columns)
view_pydot(dot)

NameError: name 'cg' is not defined

In [188]:
model= dowhy.CausalModel(
        data = df3,
        treatment = "Resource_1+Resource_2+Resource_3+Resource_4+Resource_5+Resource_6+Resource_7+Resource_8+Resource_9+Resource_10+Resource_11+Resource_12+Resource_13+Resource_14+Resource_15+Resource_16+Resource_17+Resource_18".split('+'),
        outcome = "dayspast_18",  
        graph ="digraph{Includes_subCases -> action_code_1;"+
"Includes_subCases -> action_code_10;"+
"Includes_subCases -> action_code_6;"+
"Includes_subCases -> action_code_7;"+
"Includes_subCases -> action_code_8;"+
"Includes_subCases -> action_code_9;"+
"Includes_subCases -> dayspast_6;"+
"Includes_subCases -> dayspast_7;"+
"Includes_subCases -> dayspast_8;"+
"Includes_subCases -> dayspast_9;"+
"Includes_subCases -> dayspast_10;"+
"Resource_1 -> Resource_2;"+
"Resource_1 -> action_code_1;"+
"Resource_1 -> dayspast_1;"+
"Resource_3 -> Resource_4;"+
"Resource_3 -> action_code_3;"+
"Resource_3 -> dayspast_3;"+
"Resource_10 -> Resource_11;"+
"Resource_10 -> action_code_10;"+
"Resource_10 -> dayspast_10;"+
"Resource_13 -> Resource_14;"+
"Resource_13 -> action_code_13;"+
"Resource_13 -> dayspast_13;"+
"Resource_11 -> Resource_12;"+
"Resource_11 -> action_code_11;"+
"Resource_11 -> dayspast_11;"+
"Resource_12 -> Resource_14;"+
"Resource_12 -> action_code_12;"+
"Resource_12 -> action_code_13;"+
"Resource_12 -> dayspast_12;"+
"Resource_14 -> Resource_16;"+
"Resource_14 -> action_code_14;"+
"Resource_14 -> action_code_15;"+
"Resource_14 -> dayspast_14;"+
"Resource_14 -> dayspast_15;"+
"Resource_16 -> Resource_17;"+
"Resource_16 -> action_code_16;"+
"Resource_16 -> dayspast_16;"+
"Resource_17 -> action_code_17;"+
"Resource_17 -> action_code_18;"+
"Resource_17 -> dayspast_17;"+
"Resource_2 -> Resource_4;"+
"Resource_2 -> action_code_2;"+
"Resource_2 -> action_code_3;"+
"Resource_2 -> dayspast_2;"+
"Resource_4 -> Resource_5;"+
"Resource_4 -> action_code_4;"+
"Resource_4 -> dayspast_4;"+
"Resource_5 -> Resource_6;"+
"Resource_5 -> action_code_5;"+
"Resource_5 -> dayspast_5;"+
"Resource_6 -> Resource_7;"+
"Resource_6 -> action_code_6;"+
"Resource_6 -> dayspast_6;"+
"Resource_7 -> Resource_8;"+
"Resource_7 -> action_code_7;"+
"Resource_7 -> dayspast_7;"+
"Resource_8 -> Resource_9;"+
"Resource_8 -> action_code_8;"+
"Resource_8 -> dayspast_8;"+
"Resource_9 -> action_code_10;"+
"Resource_9 -> action_code_9;"+
"Resource_9 -> dayspast_10;"+
"Resource_9 -> dayspast_9;"+
"Resource_15 -> Resource_16;"+
"Resource_15 -> action_code_15;"+
"Resource_18 -> action_code_18;"+
"SUMleges -> action_code_1;"+
"SUMleges -> action_code_10;"+
"SUMleges -> action_code_11;"+
"SUMleges -> action_code_12;"+
"SUMleges -> action_code_13;"+
"SUMleges -> action_code_14;"+
"SUMleges -> action_code_15;"+
"SUMleges -> action_code_16;"+
"SUMleges -> action_code_17;"+
"SUMleges -> action_code_18;"+
"SUMleges -> action_code_2;"+
"SUMleges -> action_code_3;"+
"SUMleges -> action_code_4;"+
"SUMleges -> action_code_5;"+
"SUMleges -> action_code_6;"+
"SUMleges -> action_code_7;"+
"SUMleges -> action_code_8;"+
"SUMleges -> action_code_9;"+
"dayspast_1 -> dayspast_2;"+
"dayspast_1 -> dayspast_3;"+
"dayspast_1 -> dayspast_4;"+
"dayspast_1 -> dayspast_5;"+
"dayspast_2 -> dayspast_6;"+
"dayspast_2 -> dayspast_7;"+
"dayspast_2 -> dayspast_8;"+
"dayspast_3 -> dayspast_6;"+
"dayspast_3 -> dayspast_7;"+
"dayspast_3 -> dayspast_8;"+
"dayspast_4 -> dayspast_6;"+
"dayspast_4 -> dayspast_7;"+
"dayspast_4 -> dayspast_8;"+
"dayspast_5 -> dayspast_6;"+
"dayspast_5 -> dayspast_7;"+
"dayspast_5 -> dayspast_8;"+
"dayspast_6 -> dayspast_9;"+
"dayspast_6 -> dayspast_10;"+
"dayspast_6 -> dayspast_11;"+
"dayspast_6 -> dayspast_12;"+
"dayspast_7 -> dayspast_9;"+
"dayspast_7 -> dayspast_10;"+
"dayspast_7 -> dayspast_11;"+
"dayspast_7 -> dayspast_12;"+
"dayspast_8 -> dayspast_9;"+
"dayspast_8 -> dayspast_10;"+
"dayspast_8 -> dayspast_11;"+
"dayspast_8 -> dayspast_12;"+
"dayspast_9 -> dayspast_15;"+
"dayspast_10 -> dayspast_15;"+
"dayspast_11-> dayspast_15;"+
"dayspast_12 -> dayspast_15;"+
"dayspast_15 -> dayspast_16;"+
"dayspast_15-> dayspast_17;"+
"dayspast_16 -> dayspast_18;"+
"dayspast_17 -> dayspast_18;"+
"requestComplete -> action_code_1;"+
"requestComplete -> action_code_10;"+
"requestComplete -> action_code_11;"+
"requestComplete -> action_code_12;"+
"requestComplete -> action_code_13;"+
"requestComplete -> action_code_14;"+
"requestComplete -> action_code_15;"+
"requestComplete -> action_code_16;"+
"requestComplete -> action_code_17;"+
"requestComplete -> action_code_18;"+
"requestComplete -> action_code_2;"+
"requestComplete -> action_code_3;"+
"requestComplete -> action_code_4;"+
"requestComplete -> action_code_5;"+
"requestComplete -> action_code_6;"+
"requestComplete -> action_code_7;"+
"requestComplete -> action_code_8;"+
"requestComplete -> action_code_9;}"
)
model.view_model()

In [189]:
identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
print(identified_estimand)

Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
                                                                              
──────────────────────────────────────────────────────────────────────────────
d[Resource₁  Resource₂  Resource₃  Resource₄  Resource₅  Resource₆  Resource₇ 

                         d                                                    
──────────────────────────────────────────────────────────────────────────────
 Resource₈  Resource₉  Resource₁₀  Resource₁₁  Resource₁₂  Resource₁₃  Resourc

                                                                   
────────────────────────────────────────────────────(E[dayspast₁₈])
e₁₄  Resource₁₅  Resource₁₆  Resource₁₇  Resource₁₈]               
Estimand assumption 1, Unconfoundedness: If U→{Resource_1,Resource_2,Resource_3,Resource_4,Resource_5,Resource_6,Resource_7,Resource_8,Resource_9,Resource_10,Resource_11,Resource_12,Resource_13,Resource_14,Resource_15,Resour

In [190]:
estimate = model.estimate_effect(identified_estimand,
        method_name="backdoor.linear_regression")
print(estimate)
print("Causal Estimate is " + str(estimate.value))

linear_regression
{'control_value': 0, 'treatment_value': 1, 'test_significance': None, 'evaluate_effect_strength': False, 'confidence_intervals': False, 'target_units': 'ate', 'effect_modifiers': ['Includes_subCases']}
*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
                                                                              
──────────────────────────────────────────────────────────────────────────────
d[Resource₁  Resource₂  Resource₃  Resource₄  Resource₅  Resource₆  Resource₇ 

                         d                                                    
──────────────────────────────────────────────────────────────────────────────
 Resource₈  Resource₉  Resource₁₀  Resource₁₁  Resource₁₂  Resource₁₃  Resourc

                                                                   
────────────────────────────────────────────────────(E[dayspast₁₈])
e₁₄  Resource₁₅  Resourc

In [191]:
refute_results = model.refute_estimate(identified_estimand, estimate,
                                       method_name="random_common_cause")
print(refute_results)

{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': False, 'target_units': 'ate', 'effect_modifiers': ['Includes_subCases']}
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': False, 'target_units': 'ate', 'effect_modifiers': ['Includes_subCases']}
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': False, 'target_units': 'ate', 'effect_modifiers': ['Includes_subCases']}
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': False, 'target_units': 'ate', 'effect_modifiers': ['Includes_subCases']}
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': False, 'target_units': 'ate', 'effect_modifiers': ['Includ

In [192]:
estimate = model.estimate_effect(identified_estimand,
        method_name="iv.linear_regression")
print(estimate)
print("Causal Estimate is " + str(estimate.value))

linear_regression
{'control_value': 0, 'treatment_value': 1, 'test_significance': None, 'evaluate_effect_strength': False, 'confidence_intervals': False, 'target_units': 'ate', 'effect_modifiers': ['Includes_subCases']}
*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: iv
Estimand expression:
 ⎡                                                                            
 ⎢            d                         ⎛            ∂                        
E⎢─────────────────────────(dayspast₁₈)⋅⎜─────────────────────────([Resource₁ 
 ⎣d[Resource₁₅  Resource₁₃]             ⎝∂[Resource₁₅  Resource₁₃]            

                                                                              
                                                                              
 Resource₂  Resource₃  Resource₄  Resource₅  Resource₆  Resource₇  Resource₈  
                                                                              

        

In [193]:
refute_results = model.refute_estimate(identified_estimand, estimate,
                                       method_name="random_common_cause")
print(refute_results)

{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': False, 'target_units': 'ate', 'effect_modifiers': ['Includes_subCases']}
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': False, 'target_units': 'ate', 'effect_modifiers': ['Includes_subCases']}
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': False, 'target_units': 'ate', 'effect_modifiers': ['Includes_subCases']}
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': False, 'target_units': 'ate', 'effect_modifiers': ['Includes_subCases']}
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': False, 'target_units': 'ate', 'effect_modifiers': ['Includ